In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import cftime

# Set the path to the NetCDF file
netcdf_file = '/Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/hw95_summary/i.e215.I2000Clm50SpGs.hw_production.02.clm2.h1.hwdaysOnly.nc'

# Set the output directory
output_dir = '/Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/hw95_summary'

summary_dir = '/Trex/case_results/i.e215.I2000Clm50SpGs.hw_production.02/research_results/summary'


In [ ]:


# Open the NetCDF file
ds = xr.open_dataset(netcdf_file)

# Select the variables we want to include
variables = ['TSA', 'TSA_U', 'TSA_R', 'TREFMXAV_R', 'HW']
ds_subset = ds[variables]

# Convert to DataFrame
df = ds_subset.to_dataframe().reset_index()

# Drop rows where TSA_U is missing
df = df.dropna(subset=['TSA_U'])

# Drop rows where TREFMXAV_R
df = df.dropna(subset=['TREFMXAV_R']) # this will remove 19850101 data

# Convert cftime to pandas datetime
def convert_cftime_to_datetime(ct):
    return pd.Timestamp(ct.year, ct.month, ct.day)

df['time'] = df['time'].apply(convert_cftime_to_datetime)


loc_id_path = os.path.join(summary_dir, 'location_IDs.nc')
location_ds = xr.open_dataset(loc_id_path)
location_df = location_ds.to_dataframe().reset_index()

# Merge the location_df with the local_hour_adjusted_df
df = pd.merge(df, location_df, on=['lat', 'lon'], how='left')

# Sort the DataFrame
df = df.sort_values(['location_ID', 'time'])

# Save the DataFrame as a feather file
output_file = os.path.join(output_dir, 'hw_data.feather')
df.to_feather(output_file)

print(f"Data has been saved to {output_file}")

# Print some information about the resulting DataFrame
print("\nDataFrame Info:")
print(df.info())

print("\nFirst few rows of the DataFrame:")
print(df.head())

# Print unique lat-lon pairs
unique_locations = df[['lat', 'lon']].drop_duplicates()
print(f"\nNumber of unique lat-lon pairs: {len(unique_locations)}")



In [ ]:
# Count unique lat-lon pairs where HW = 1
hw_locations = df[df['HW'] == 1][['lat', 'lon']].drop_duplicates()
hw_location_count = len(hw_locations)

print(f"Number of unique lat-lon pairs with HW = 1: {hw_location_count}")

## add location_ID 

In [ ]:


df

In [ ]:
# find row NaN on TREFMAXV_R
t_nan_dates =pd.DataFrame(df[df['TREFMXAV_R'].isnull()])


In [ ]:
t_nan_dates.info()

In [ ]:
import pandas as pd

result = t_nan_dates.groupby('location_ID').filter(lambda x: len(x) == 1).groupby('location_ID').agg(
    first_time=('time', 'first'),
    count=('time', 'size')
)

result = result[result['first_time'] > '1985-01-01']
print(result)

In [ ]:
import duckdb 

nan_dates = duckdb.query(
    """
    PRAGMA threads=4;
    SELECT location_ID, COUNT(*) AS count, min(time) 
    FROM df 
    Where time > '1985-01-01' and TREFMXAV_R is NULL 
    GROUP BY location_ID  
    having count > 1
    """
).to_df()

In [ ]:
df[(df['location_ID'] == 14825) & df['TREFMXAV_R'].isnull()]


In [ ]:
duckdb.query(
    """
    SELECT location_ID, COUNT(*) AS count
    FROM df 
    group by location_ID
    order by count desc
    """
).to_df()

In [ ]:
#
df = df.dropna(subset=['TREFMXAV_R'])